<a href="https://colab.research.google.com/github/RealMyeong/AIFFELTHON/blob/main/JinMyeong/BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

%matplotlib inline
# 한글 지원 폰트
plt.rc('font', family='NanumBarunGothic')

# 마이너스 부호 
plt.rcParams['axes.unicode_minus'] = False

import re
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from soynlp.word import WordExtractor

data = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_4000_mk2.csv')

In [44]:
data

,non_label_sentence,label_sentence,class,binary_class
0,금연한지 얼마나 됐지\n1년 넘었지\n인간승리다 진짜\n그러게 대신 확실히 입이 ...,1:금연한지 얼마나 됐지\n2:1년 넘었지\n1:인간승리다 진짜\n2:그러게 대신 ...,일반 대화,일반 대화
1,너 여자친구 등산 좋아 하냐\n걷는 건 좋아하는데 등산을 좋아하는지는 모르겠네\n너...,1:너 여자친구 등산 좋아 하냐\n2:걷는 건 좋아하는데 등산을 좋아하는지는 모르겠...,일반 대화,일반 대화
2,오늘도 에 일이 없어\n널널히 일하면 되겠다\n응 컴퓨터도 고장나서 더 할 일이 없...,1:오늘도 에 일이 없어\n2:널널히 일하면 되겠다\n1:응 컴퓨터도 고장나서 더 ...,일반 대화,일반 대화
3,얌 그 소식은 안당 배구 선수\n어 아주 세간의 난리더라\n걔 자기 언니한테 욕한거...,1:얌 그 소식은 안당 배구 선수\n2:어 아주 세간의 난리더라\n1:걔 자기 언니...,일반 대화,일반 대화
4,어제 날씨 좀 따뜻하던데 하늘이 많이 어둡더라\n어제 안개 낀 거 처럼 뿌옇더라\n...,1:어제 날씨 좀 따뜻하던데 하늘이 많이 어둡더라\n2:어제 안개 낀 거 처럼 뿌옇...,일반 대화,일반 대화
...,...,...,...,...
7865,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,1:준하야 넌 대가리가 왜이렇게 크냐\n2:내 머리가 뭐\n1:밥먹으면 대가리만 크...,기타 괴롭힘 대화,공격 대화
7866,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,갈취 대화,공격 대화
7867,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,직장 내 괴롭힘 대화,공격 대화
7868,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,1:누구맘대로 여기서 장사하래\n2:이게 무슨일입니까\n1:남의 구역에서 장사하려면...,갈취 대화,공격 대화


In [45]:
# # 불용어 정의
# stop_data = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/한글불용어.csv')
# stop_data
# stopwords = []
# for i in stop_data['불용어']:
#   i = i.split()
#   stopwords += i

# stopwords
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '만', '게임', '겜', '되', '음', '면']

In [46]:
sentences = data.non_label_sentence
sentences = [sen for sen in sentences]

from soynlp.tokenizer import LTokenizer

word_extractor = WordExtractor(
    min_frequency=100, # example
    min_cohesion_forward=0.05,
    min_right_branching_entropy=0.0
)

word_extractor.train(sentences)
words = word_extractor.extract()

cohesion_score = {word:score.cohesion_forward for word, score in words.items()}
tokenizer = LTokenizer(scores=cohesion_score)

training was done. used memory 3.469 Gb
all cohesion probabilities was computed. # words = 1496
all branching entropies was computed # words = 36252
all accessor variety was computed # words = 36252


In [47]:
data['tokenized'] = data['non_label_sentence'].apply(tokenizer.tokenize)
data['tokenized'] = data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
data

,non_label_sentence,label_sentence,class,binary_class,tokenized
0,금연한지 얼마나 됐지\n1년 넘었지\n인간승리다 진짜\n그러게 대신 확실히 입이 ...,1:금연한지 얼마나 됐지\n2:1년 넘었지\n1:인간승리다 진짜\n2:그러게 대신 ...,일반 대화,일반 대화,"[금연한지, 얼마, 나, 됐지, 1년, 넘었지, 인간승리다, 진짜, 그러게, 대신,..."
1,너 여자친구 등산 좋아 하냐\n걷는 건 좋아하는데 등산을 좋아하는지는 모르겠네\n너...,1:너 여자친구 등산 좋아 하냐\n2:걷는 건 좋아하는데 등산을 좋아하는지는 모르겠...,일반 대화,일반 대화,"[너, 여자친구, 등산, 좋아, 하냐, 걷는, 건, 좋아하는, 데, 등산을, 좋아하..."
2,오늘도 에 일이 없어\n널널히 일하면 되겠다\n응 컴퓨터도 고장나서 더 할 일이 없...,1:오늘도 에 일이 없어\n2:널널히 일하면 되겠다\n1:응 컴퓨터도 고장나서 더 ...,일반 대화,일반 대화,"[오늘, 일이, 없어, 널널히, 일하, 되겠, 응, 컴퓨터도, 고장나서, 더, 할,..."
3,얌 그 소식은 안당 배구 선수\n어 아주 세간의 난리더라\n걔 자기 언니한테 욕한거...,1:얌 그 소식은 안당 배구 선수\n2:어 아주 세간의 난리더라\n1:걔 자기 언니...,일반 대화,일반 대화,"[얌, 그, 소식은, 안당, 배구, 선수, 어, 아주, 세간의, 난리, 더라, 걔,..."
4,어제 날씨 좀 따뜻하던데 하늘이 많이 어둡더라\n어제 안개 낀 거 처럼 뿌옇더라\n...,1:어제 날씨 좀 따뜻하던데 하늘이 많이 어둡더라\n2:어제 안개 낀 거 처럼 뿌옇...,일반 대화,일반 대화,"[어제, 날씨, 좀, 따뜻하던데, 하늘이, 많이, 어둡더라, 어제, 안개, 낀, 거..."
...,...,...,...,...,...
7865,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,1:준하야 넌 대가리가 왜이렇게 크냐\n2:내 머리가 뭐\n1:밥먹으면 대가리만 크...,기타 괴롭힘 대화,공격 대화,"[준하야, 넌, 대가리가, 왜, 이렇게, 크냐, 내, 머리, 뭐, 밥, 먹으면, 대..."
7866,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,갈취 대화,공격 대화,"[내가, 지금, 너, 아들, 김길준, 데리고, 있어, 살리고, 싶으, 계좌, 1억만..."
7867,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,직장 내 괴롭힘 대화,공격 대화,"[나는, 씨, 같은, 사람, 보면, 참, 신기, 하더라, 어떻게, 저렇게, 살지, ..."
7868,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,1:누구맘대로 여기서 장사하래\n2:이게 무슨일입니까\n1:남의 구역에서 장사하려면...,갈취 대화,공격 대화,"[누구, 맘대로, 여기, 서, 장사하래, 이게, 무슨, 일입니까, 남의, 구역에서,..."


In [48]:
conversation = data['tokenized'].values

tokenizer_tf = Tokenizer()
tokenizer_tf.fit_on_texts(conversation)
len(tokenizer_tf.word_index)

81173

In [49]:
threshold = 2
total_cnt = len(tokenizer_tf.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer_tf.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print('전체 단어 집합에서 희귀 단어를 뺀 크기 : ', total_cnt - rare_cnt)
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 81173
등장 빈도가 1번 이하인 희귀 단어의 수: 55278
전체 단어 집합에서 희귀 단어를 뺀 크기 :  25895
단어 집합에서 희귀 단어의 비율: 68.09899843544036
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 9.616508705273267


In [50]:
vocab_sie = total_cnt - rare_cnt
tokenizer_tf = Tokenizer(vocab_size) 
tokenizer_tf.fit_on_texts(conversation)
X_data = tokenizer_tf.texts_to_sequences(conversation)

In [51]:
padded = pad_sequences(X_data)
print(padded)

[[    0     0     0 ...  4968    43  1765]
 [    0     0     0 ... 25914  1079 25915]
 [    0     0     0 ...    74  2723  5555]
 ...
 [    0     0     0 ...     5     1   309]
 [    0     0     0 ...   237  8778   401]
 [    0     0     0 ...   422   123 25841]]


In [52]:
np.shape(padded)

(7870, 181)

In [53]:
train_data = padded
train_label = data['class']

In [54]:
labels = {'협박 대화':0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '기타 괴롭힘 대화':3, '일반 대화':4}

In [55]:
train_label = train_label.apply(lambda x: labels[x])
train_label = pd.get_dummies(train_label)
print(train_label)

      0  1  2  3  4
0     0  0  0  0  1
1     0  0  0  0  1
2     0  0  0  0  1
3     0  0  0  0  1
4     0  0  0  0  1
...  .. .. .. .. ..
7865  0  0  0  1  0
7866  0  1  0  0  0
7867  0  0  1  0  0
7868  0  1  0  0  0
7869  0  0  1  0  0

[7870 rows x 5 columns]


In [56]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(train_data, train_label, test_size=0.2, random_state=22, stratify=train_label)
valid_X, test_X, valid_Y, test_Y = train_test_split(test_X, test_Y, test_size=0.5, random_state=22, stratify=test_Y)

print(len(train_X), len(valid_X), len(test_X))
print(len(train_Y), len(valid_Y), len(test_Y))

6296 787 787
6296 787 787


In [57]:
embedding_dim = 1024
label_size = len(labels)
hidden_units = 128

# BiLSTM
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_shape=(None,)))
# model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units, return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units, return_sequences=True, dropout=0.4)))
# model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units, return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units, return_sequences=True, dropout=0.4)))
# model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units, return_sequences=True)))
# model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units, return_sequences=True, dropout=0.4)))
# model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units//2)))
model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units//2, dropout=0.4)))
model.add(tf.keras.layers.Dense(vocab_size, activation='relu'))
# model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(label_size, activation='softmax'))

# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_shape=(None,)))
# model.add(tf.keras.layers.LSTM(hidden_units, return_sequences=True, dropout=0.4))
# model.add(tf.keras.layers.LSTM(hidden_units, return_sequences=True, dropout=0.4))
# model.add(tf.keras.layers.LSTM(hidden_units//2, dropout=0.4))
# model.add(tf.keras.layers.Dense(vocab_size, activation='relu'))
# # model.add(tf.keras.layers.Dropout(0.4))
# model.add(tf.keras.layers.Dense(label_size, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 1024)        30720000  
                                                                 
 bidirectional (Bidirectiona  (None, None, 256)        1180672   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 256)        394240    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 30000)             3870000   
                                                      

In [58]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
mc = ModelCheckpoint(filepath='/content/sample_data/best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs=10

history = model.fit(train_X,
                    train_Y,
                    epochs=epochs,
                    batch_size=256,
                    callbacks=[es, mc],
                    validation_data=(valid_X, valid_Y),
                    verbose=1)

Epoch 1/10
25/25 [==============================] - ETA: 0s - loss: 1.0972 - accuracy: 0.5534
Epoch 1: val_accuracy improved from -inf to 0.64041, saving model to /content/sample_data/best_model.h5
25/25 [==============================] - 26s 403ms/step - loss: 1.0972 - accuracy: 0.5534 - val_loss: 0.7839 - val_accuracy: 0.6404
Epoch 2/10
25/25 [==============================] - ETA: 0s - loss: 0.5635 - accuracy: 0.7506
Epoch 2: val_accuracy improved from 0.64041 to 0.80432, saving model to /content/sample_data/best_model.h5
25/25 [==============================] - 8s 306ms/step - loss: 0.5635 - accuracy: 0.7506 - val_loss: 0.4518 - val_accuracy: 0.8043
Epoch 3/10
25/25 [==============================] - ETA: 0s - loss: 0.2189 - accuracy: 0.9215
Epoch 3: val_accuracy improved from 0.80432 to 0.87421, saving model to /content/sample_data/best_model.h5
25/25 [==============================] - 8s 309ms/step - loss: 0.2189 - accuracy: 0.9215 - val_loss: 0.3624 - val_accuracy: 0.8742
Epoch 

In [59]:
loaded_model = load_model('/content/sample_data/best_model.h5')
pred = loaded_model.predict(test_X)
print("테스트 정확도: %.4f" % (loaded_model.evaluate(test_X, test_Y)[1]))

25/25 [==============================] - 3s 27ms/step - loss: 0.5068 - accuracy: 0.8806
테스트 정확도: 0.8806


In [60]:
from sklearn.metrics import classification_report

pred_label = []
test_label = []
test_index = []

for i in range(len(pred)):
  pred_label.append((str)(np.argmax(pred[i])))
  test_label.append((str)(labels[data['class'][test_Y.index[i]]]))
  test_index.append(test_Y.index[i])

print(classification_report(test_label, pred_label))

              precision    recall  f1-score   support

           0       0.85      0.72      0.78        89
           1       0.81      0.81      0.81        97
           2       0.83      0.81      0.82        96
           3       0.72      0.79      0.75       105
           4       0.96      0.97      0.97       400

    accuracy                           0.88       787
   macro avg       0.83      0.82      0.83       787
weighted avg       0.88      0.88      0.88       787



In [61]:
lidx2labels = {str(v) : k for k, v in labels.items()}

In [62]:
test_index
# 잘못 분류한 부분 출력

error_idx = []
error_label = []
error_data_idx = []

for idx, val in enumerate(pred_label):
  if val != test_label[idx]:
    error_data_idx.append(test_index[idx])
    error_idx.append(idx)
    error_label.append(val)

In [63]:
len(error_idx)

94

In [64]:
error = test_X[error_idx]

In [65]:
error_text = tokenizer_tf.sequences_to_texts(error)

In [66]:
result = []

for i in range(len(error_idx)):
  result.append([data['non_label_sentence'][error_data_idx[i]], lidx2labels[error_label[i]], data['class'][error_data_idx[i]]])

result

[['제가 언제 그쪽을 봤다고 그러세요\n이게 야 그럼 내가 눈깔이 삐었냐  아까부터 네가 계속 쳐다봤잖아\n아니라는데 왜 자꾸 그러세요\n아니긴 뭐가 아니야  핸드폰 내놔 바 몰래 찍었을 수도 있어\n제발 그러지 마세요 아니라는데 왜 그러세요\n그럼 왜 마주치자마자 고개 돌리는데\n제발 믿어주세요  저 그쪽 안 봤어요  그리고 그쪽 제 스타일 아니에요\n뭐  이게 진짜  돌았나  언제 봤다고 그쪽이래\n제발 부탁이니까 저리 좀 가세요  술도 곱게 마셔야지\n너 이리 안 와',
  '협박 대화',
  '기타 괴롭힘 대화'],
 ['저기 김 씨\n아 네 대표님\n혹시 주말에 시간 돼\n네  무슨 일로 그러세요\n아 그게 내가 이번 주에 다른 지역을 가야 하는데\n네네\n차가 고장 나서 말이야\n아 그러세요  제가 시간이 안돼서 죄송합니다\n뭐  아니 차가 고장 났다는데 너무 매정한 거 아니야  너 각오해 감히 대표 말을 거절해\n죄송합니다',
  '기타 괴롭힘 대화',
  '직장 내 괴롭힘 대화'],
 ['아 이제 결혼 준비 거의 다 끝나간다\n벌써  그럼 이제 뭐뭐 남았어\n나 이제 신혼집 꾸미기만 하면 돼\n오 제일 재밌는게 남았네\n근데 신경쓸게 한 두개가 아니라 힘들어\n가전용품들은 다 샀어\n응응 다 사서 배송만 오면 돼\n잘했네 내가 뭐 사줄 건 없어\n나 블루투스 스피커 갖고 싶어 그거 선물해줘\n알겠어 좋은 걸로 사서 보내줄게',
  '갈취 대화',
  '일반 대화'],
 ['우리 헤어져\n뭐  왜 갑자기 그런 소리를 해\n너 너무 집착이 심해  못 견디겠어\n내가 언제 그랬는데  오히려 믿음 안 준 건 너잖아\n그래  다 내 잘못이니까 헤어지자\n야  생각을 하고 얘기를 해  너 나랑 쉽게 헤어질 수 있을 것 같아\n이젠 협박까지 하시겠다\n너 내 핸드폰에 너 관련된 거 엄청 많아  안 무섭냐\n무슨 얘기 하는 거야\n넌 나랑 헤어지는 순간 인터넷에 다 공개해버릴 거라고  그러니까 나랑 계속 만나는 게 좋을 거야',
  '기타 괴롭힘 대화',
 

In [67]:
result_df = pd.DataFrame(result, columns=['text', 'predict', 'true'])

In [68]:
result_df

,text,predict,true
0,제가 언제 그쪽을 봤다고 그러세요\n이게 야 그럼 내가 눈깔이 삐었냐 아까부터 네...,협박 대화,기타 괴롭힘 대화
1,저기 김 씨\n아 네 대표님\n혹시 주말에 시간 돼\n네 무슨 일로 그러세요\n아...,기타 괴롭힘 대화,직장 내 괴롭힘 대화
2,아 이제 결혼 준비 거의 다 끝나간다\n벌써 그럼 이제 뭐뭐 남았어\n나 이제 신...,갈취 대화,일반 대화
3,우리 헤어져\n뭐 왜 갑자기 그런 소리를 해\n너 너무 집착이 심해 못 견디겠어...,기타 괴롭힘 대화,갈취 대화
4,저기요 집 앞에 담배꽁초 버리지 마세요 저번에도 쓰레기봉투에 옮겨붙어서 불날 뻔했잖...,기타 괴롭힘 대화,협박 대화
...,...,...,...
89,지금 당장 나와 합의하고 해결 보자\n어떻게 그래 이제껏 내가 당한 게 있는데 이렇...,기타 괴롭힘 대화,협박 대화
90,아줌마 손에 든 거 뭐예요\n장바구니인데요\n아니 장바구니 인건 아는데 뭐 들었어요...,기타 괴롭힘 대화,갈취 대화
91,저기 기업 보안부서죠\n네 그런데요\n제가 당신 회원들 개인 정보를 전부 해킹했는...,일반 대화,갈취 대화
92,야 재형아\n미친 새끼가 누구 보고 재형이래 더럽네\n하\n하 야 새끼야 내가 ...,직장 내 괴롭힘 대화,기타 괴롭힘 대화


In [69]:
# True : 협박 대화
# 샘플 확인
threat_sample = result_df[result_df['true']=='협박 대화'].sample(2).index
print('text 1')
print('----'*20)
print(result_df['text'][threat_sample[0]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[0]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[0]])
print('----'*20)
print('text 2')
print('----'*20)
print(result_df['text'][threat_sample[1]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[1]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[1]])

text 1
--------------------------------------------------------------------------------
손들고 움직이지 마
이보시오 이러지 마시고 말로 합시다
이 칼 안 보여  움직이지 마
알겠습니다
좋아 너는 거기 꼼짝 말고 거기 네놈이 돈 되는 것 다 쓸어 담아 가지고 와
제 제가요
그래 너  허튼수작 부리면 살아서 못 나갈 줄 알아
아 알겠습니다
빨리
빨리하고 있습니다

예측 라벨 :  갈취 대화

실제 라벨 :  협박 대화
--------------------------------------------------------------------------------
text 2
--------------------------------------------------------------------------------
이번에 뇌물 받으신 거 다 알고 있습니다
그래서 어쩌려고
여기 녹취록이랑 출금 내역
원하는 게 뭐야
저희 이번에 아파트 건설 들어가는 거 아시죠
시청 옆 부지 말하는 건가
이번에 규제 하나 때문에 몇억이 더 들게 생겼거든요
그냥 넘어가 달란 이야기인가
그렇지 않으면 지금 바로 이것들 기자한테 넘기고 죽일 수밖에 없어요
알겠네  받아들이겠어

예측 라벨 :  기타 괴롭힘 대화

실제 라벨 :  협박 대화


In [70]:
# True : 갈취 대화
# 샘플 확인
threat_sample = result_df[result_df['true']=='갈취 대화'].sample(2).index
print('text 1')
print('----'*20)
print(result_df['text'][threat_sample[0]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[0]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[0]])
print('----'*20)
print('text 2')
print('----'*20)
print(result_df['text'][threat_sample[1]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[1]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[1]])

text 1
--------------------------------------------------------------------------------
해당 업체에 현수막 주문을 하려고 합니다
주문을 하시려면 결제를 먼저 해주셔야 하십니다
우리가 거기 거래 한지 10년이 넘었는데 외상으로 안됩니까
이건  내 방침이라 죄송합니다
아니 해당 업체가 이런 식이면 앞으로 거래는 여기서 못하죠
방침인데 왜 거래 이야기까지 나오십니까
방침이던 뭐든 단골한테 외상 한 번 해줄 수 있는 거죠
저희가 외상으로 못 받은 돈들이 좀 있어서요  죄송합니다
누가 떼어먹는데요  준다니깐요 나중에
알겠습니다

예측 라벨 :  직장 내 괴롭힘 대화

실제 라벨 :  갈취 대화
--------------------------------------------------------------------------------
text 2
--------------------------------------------------------------------------------
어이  네가 그 새로 온 찌질이 전학생이지
누  구세요
나 2학년 선배  야 너 가방 예뻐 보인다  안에 좀 보자
네   제가 지금 빨리 집에 가봐야 해서
아니 잠깐만 보자니깐  빨리
아 별거 없습니다  진짜로요
너 한대 맞기 싫으면 당장 가방 내놔
네
털어서 뭐 나오기만 해봐라 진짜
별거 없어요 진짜

예측 라벨 :  기타 괴롭힘 대화

실제 라벨 :  갈취 대화


In [71]:
# True : 직장 내 괴롭힘 대화
# 샘플 확인
threat_sample = result_df[result_df['true']=='직장 내 괴롭힘 대화'].sample(2).index
print('text 1')
print('----'*20)
print(result_df['text'][threat_sample[0]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[0]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[0]])
print('----'*20)
print('text 2')
print('----'*20)
print(result_df['text'][threat_sample[1]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[1]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[1]])

text 1
--------------------------------------------------------------------------------
오늘 점심은 내가 쏜다  같이 가자
좋아요  너무 신나요
뭐 먹을까  회 먹으러 갈까
아 그런데 한 대리가 회를 못 먹어요
아 죄송합니다
아 안 때리는 못 먹는 게 너무 많네 어쩌지
아 저는 괜찮습니다  같이 가서 다른 걸 먹어도 됩니다
아냐 안 때리는 회도 못 먹는데 왜 굳이 같이 가려 그래
우리는 회 다 좋아해서 괜찮은데 불편하지 않겠어
아  그런가요
어어 점심 대충 알아서 먹고 내가 말한 보고서 얼른 올려줘 우리는 회 먹고 올게
아  네

예측 라벨 :  기타 괴롭힘 대화

실제 라벨 :  직장 내 괴롭힘 대화
--------------------------------------------------------------------------------
text 2
--------------------------------------------------------------------------------
김 선생님 옷이 예쁘네요  새로 사셨나 봐요
아
오늘은 수업시간에 조는 남학생들이 없어서 수업이 잘 되겠어요
네
그래도 다른 걸로 학생들을 집중시켜야지 그래서야 되겠어요
교감 선생님  말씀이 좀 지나치십니다
나야 보기 좋은데  김 선생님이 걱정돼서 그러지요  한참 예민할 나이들이잖아  우리 애들이
어떤 게 문젠지 정확히 말씀해 주시면 고치겠습니다  치마 기장도 길고 몸에 붙는 옷도 아닌데요
색이 야하잖아요 색이  빨간색이 뭡니까  교사가
그런 시대착오적인 생각을 하시다뇨  요즘 그렇게 생각하는 사람이 어디 있습니까
쯧쯧  교산리 학생인지 교산리 어디 종업원인지 알 수가 없다니까  쯧쯧

예측 라벨 :  일반 대화

실제 라벨 :  직장 내 괴롭힘 대화


In [72]:
# True : 기타 괴롭힘 대화
# 샘플 확인
threat_sample = result_df[result_df['true']=='기타 괴롭힘 대화'].sample(2).index
print('text 1')
print('----'*20)
print(result_df['text'][threat_sample[0]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[0]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[0]])
print('----'*20)
print('text 2')
print('----'*20)
print(result_df['text'][threat_sample[1]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[1]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[1]])

text 1
--------------------------------------------------------------------------------
야 재형아
미친 새끼가 누구 보고 재형이래 더럽네
하
하   야 새끼야 내가 더 기분 나빠 존재 자체가 민폐인 새끼야
야 재형이 보고 새끼라고 하지 마 새끼란 욕이 불쌍해
그래 내가 미안하니까 이제 그만해 잘못했어
그래 야 현우야 작작하자 괜히 심하게 건들다가 큰일 날 것 같으니까 나중에 적당히 건들자
아니 웃긴 새끼네 야 날 나쁜 사람으로 만드냐 그리고 네가 뭔데 명령을 해
야 현우야 적당히 하자니까 저 더러운 놈 잘못 건들면 우리만 손해야
착한 내가 참는다
야 적당히 하라고 내가 뭔 죄를 졌길래 계속 그러는데 작작해
이 새끼가 미쳤구나 하교 시간 때 보자 먼지 나도록 맞을 준비하고
이 새끼 중학교 생활 망했네
너 말이야 사람 잘못 건들었어 성적표 점수도 내려갈 준비하고

예측 라벨 :  직장 내 괴롭힘 대화

실제 라벨 :  기타 괴롭힘 대화
--------------------------------------------------------------------------------
text 2
--------------------------------------------------------------------------------
아우 아줌마  여기 청소 제대로 하시라 했잖아요
죄송합니다
죄송하다 말만 하지 말고 제발 좀 제대로 좀 하세요
네  알겠습니다
아는 분 소개로 왔으면 그분 얼굴에 먹칠하지 마시고 제대로 하세요 마지막 경고
네  죄송합니다
기분 나빠요  기분 나쁘면 이 집에서 당장 나가세요
아닙니다

예측 라벨 :  직장 내 괴롭힘 대화

실제 라벨 :  기타 괴롭힘 대화


In [73]:
# True : 일반 대화
# 샘플 확인
threat_sample = result_df[result_df['true']=='일반 대화'].sample(2).index
print('text 1')
print('----'*20)
print(result_df['text'][threat_sample[0]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[0]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[0]])
print('----'*20)
print('text 2')
print('----'*20)
print(result_df['text'][threat_sample[1]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[1]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[1]])

text 1
--------------------------------------------------------------------------------
주말에 또 결혼식 가야 한다
누구 요즘 코로나 때문에 미루는 사람도 많던데
같이 일하는 사람 코로나라도 결혼할 사람은 다하네
하긴 코로나 아니라도 내가 결혼할 수 있는 거도 아니고 ㅋ
결혼식 할 때마다 축의금 때문에 돌겠네
보통 얼마냄 기준 같은 거 있음
5만원도 이제는 손 부끄럽던데 그나저나 이때까지 낸 축의금 돌려 받겠냐고
요전에 얘기했던 사람이랑은 잘 만나고 있나 곧 돌려받는 거 아니가 하하
모르겠다 만나기는 하는데
모르면 어떡하노 전에 듣기로는 괜찮은 사람인 거 같더만
저번에 내 친구가 나보고 남성 편력이라 그랬거든
엥 그런가 내가 보기엔 아닌 거 같은데
몰라 누굴 만나도 이래서 싫고 저래서 싫고 결혼 못할 거라고
그래도 이래저래 만나다 보면 맞는 사람 만나겠지 하하
이제 3개월 됐고 잘해주기도 잘해주는데 뭔가 모르겠다
어떤 부분이 신경 쓰이는데 공부 좀 해 놔야겠다 ㅋ

예측 라벨 :  직장 내 괴롭힘 대화

실제 라벨 :  일반 대화
--------------------------------------------------------------------------------
text 2
--------------------------------------------------------------------------------
어느계절 좋아하세요
전 봄 가을이 좋아요
그 중 하나 고르면요
봄이요
왜요
모든 생물이 소생하기 때문에요
전 가을이 좋아요
왜요
자연이 아름답게 변하니까요
가을 단풍이 아름답기는 하죠
봄은 나른해져서 싫어해요
그러시군요  나른하긴 해요

예측 라벨 :  직장 내 괴롭힘 대화

실제 라벨 :  일반 대화


# test

In [74]:
test_file_path = '/content/drive/MyDrive/AIFFELTHON/TUNiB/test.json'
with open(test_file_path, mode='rt', encoding='utf-8') as f:
    test_dataset = pd.read_json(f)

test_data = test_dataset.transpose()
test_data

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
...,...
t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...
t_496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...
t_497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...
t_498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...


In [75]:
import re


def remove_punctuation(x):
  x = x.strip()
  x = re.sub("[^가-힣0-9]+", " ", x)
  x = re.sub("[ ]+", " ", x)
  x = x.strip()
  return x

test_data["text"] = test_data['text'].apply(lambda x : remove_punctuation(x))
test_data.head()

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001,우리팀에서 다른팀으로 갈 사람 없나 그럼 영지씨가 가는건 어때 네 제가요 그렇지 2...
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요 제대로 좀 하지 네 똑바로 좀 하지 행실...
t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
t_004,아무튼 앞으로 니가 내 와이파이야 응 와이파이 온 켰어 반말 주인님이라고도 말해야지...


In [76]:
sen = test_data['text']
sen

t_000    아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001    우리팀에서 다른팀으로 갈 사람 없나 그럼 영지씨가 가는건 어때 네 제가요 그렇지 2...
t_002    너 오늘 그게 뭐야 네 제가 뭘 잘못했나요 제대로 좀 하지 네 똑바로 좀 하지 행실...
t_003    이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
t_004    아무튼 앞으로 니가 내 와이파이야 응 와이파이 온 켰어 반말 주인님이라고도 말해야지...
                               ...                        
t_495    미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요 네 합니다 보고서를 ...
t_496    교수님 제 논문에 제 이름이 없나요 아 무슨 논문말이야 지난 번 냈던 논문이요 그거...
t_497    야 너 네 저요 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이니 진...
t_498    야 너 빨리 안 뛰어와 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다가...
t_499    엄마 저 그 돈 안해주시면 정말 큰일나요 이유도 말하지 않고 몇번째니 경민아 엄마 ...
Name: text, Length: 500, dtype: object

In [77]:
sentences = [sen for sen in test_data['text']]
sentences = [tokenizer.tokenize(sen) for sen in sentences]
sequences = tokenizer_tf.texts_to_sequences(sentences)
padded = pad_sequences(sequences)

In [78]:
np.shape(padded)

(500, 175)

In [79]:
pred = loaded_model.predict(padded)

from sklearn.metrics import classification_report

pred_label = []

for i in range(len(pred)):
    pred_label.append((str)(np.argmax(pred[i])).zfill(2))
    
pred_label[:10]

16/16 [==============================] - 2s 28ms/step


['01', '02', '02', '02', '04', '00', '00', '01', '04', '01']

In [80]:
submission = test_data.assign(CLASS=pred_label)
submission = submission.rename(columns={'CLASS':'class'})

In [81]:
submission.drop(['text'], axis=1, inplace=True)
submission = submission.transpose()
submission

,t_000,t_001,t_002,t_003,t_004,t_005,t_006,t_007,t_008,t_009,...,t_490,t_491,t_492,t_493,t_494,t_495,t_496,t_497,t_498,t_499
class,01,02,02,02,04,00,00,01,04,01,...,03,04,00,01,01,02,02,01,02,00


In [82]:
# import json

# submission_file_path = '/content/drive/MyDrive/AIFFELTHON/TUNiB/submission/submission_LSTM_4000.json'
# result = submission.to_json(submission_file_path)

# with open(submission_file_path) as f:
#     parsed = json.load(f)

# with open(submission_file_path, 'w') as f:
#     json.dump(parsed, f, indent=4)